In order to calculate WTD from logger data we do the following:
1. Adjust the measured pressure by negating atmospheric pressure.
2. Convert the measured pressure from mmHg to mmH2O using multiplication by 13.5951. This gives us (f) in the diagram below.
3. Use the equations in the diagram below to convert field measurements to water table depth.


<div>
<img src="logger_diagram.png" width="500"/>
</div>

In [1]:
#IMPORT LIBRARIES
import os
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
# MAKE A DIRECTORY TO EXPORT PROCESSED WATER TABLE TO
try:
    os.mkdir('EXPORT')
except FileExistsError:
    pass

In [3]:
#DEFINE CONSTANTS

#The following constant converts mmHg to mmH20
HG_TO_H2O = 13.5951 #This is based on the pressure of 1 mm of H2O at 4C 
# This value will vary slightly with temperature but we are ignoring that

#Dipwell lip to logger mount (mm)
A = 20
#Dipwell lip to peat surface (mm)
B = {'FF':{'A':63,'B':59,'C':80,'D':64,'E':75,'F':62,'G':77,'H':55,'I':180},
     'Mig':50,
     'MH':{'RES':100,'NN':100,'DAM':50}}
#Length of logger wire (mm)
#Add 130 to Migneint to compensate for mismeasurement of wire when installing
C = {'FF':{'A':980,'B':970,'C':975,'D':980,'E':980,'F':980,'G':850,'H':985,'I':835},
     'Mig':{'NN1':880+130,'RES1':880+130,'RES2':880+130,'DAM1':880+130,'DAM2':880+130,'DAM3':880+130},
     'MH':880}

#Define the column titles in the logger dataframe
columnTitles = ['ID','Site','Region','Datetime','Date','Hour','Day','Week','Month','Quarter','Year','Temperature(Logger)','mmHg(Logger)','mmH2O','A','B','C','E','WTD(mm)']
#Define the order of the column titles in the final dataframe 
columnOrder = ['ID','Site','Region','Datetime','Date','Hour','Day','Week','Month','Quarter','Year','Temperature(Logger)','Temperature(Baro)','mmHg(Logger)','mmHg(Baro)','mmH2O','A','B','C','E','WTD(mm)']

In [4]:
#This is a dictionary that I can store each processed dataset in 
analysisDict = {}

In [5]:
#Read the barometer data 
MH_Baro = pd.read_csv('CSV//MH_BARO_1_SN918186_2023-02-24_13-03-16-660.csv', skip_blank_lines = True, engine = 'python')
MH_Baro['Date and Time'] = pd.DatetimeIndex(MH_Baro['Date and Time'],dayfirst=True).round("H")
MH_Baro.columns = ['Datetime','Seconds','mmHg(Baro)', 'Temperature(Baro)']
Mig_Baro = pd.read_csv('CSV//Mig_Baro_1_925338_2023-02-23_12-51-52-347.csv', skip_blank_lines = True, engine = 'python')
Mig_Baro['Date and Time'] = pd.DatetimeIndex(Mig_Baro['Date and Time'],dayfirst=True).round("H")
Mig_Baro.columns = ['Datetime','Seconds','mmHg(Baro)', 'Temperature(Baro)']
FF_Baro = pd.read_csv('CSV//FF_VuSitu_2020-02-18_12-00-00_REFAP_Log_Air reference logger.csv', skip_blank_lines = True, engine = 'python', skiprows = 27)
FF_Baro.columns = ['Datetime','mmHg(Baro)', 'Temperature(Baro)']
FF_Baro = FF_Baro.iloc[:-3,:] #remove empty rows
FF_Baro['Datetime'] = pd.DatetimeIndex(FF_Baro['Datetime'],dayfirst=True).round("H")

In [6]:
#Process the barometer data
for f in os.listdir('CSV'):
    #only parse .csv files
    if f.endswith('.csv'):
        #ignore baro data
        if 'Baro' in f or 'BARO'in f or 'REFAP' in f:
            continue 
        # Set up dataframe in analysisDict
        analysisDict[f] = pd.DataFrame(columns=columnTitles)
        # Handle Different site data 
        site = f.split('_')[0]
        region = f.split('_')[1]
        pos = f.split('_')[2]
        uqid = site + region + pos
        # read in dataset
        # forsinard flows (crocach) is in a different format so needs to be read separately 
        if site == 'FF':
            #skip to june 2021 (to speed the processing up), in order to include the whole dataset set skiprows to 27 
            df = pd.read_csv('CSV//'+f, skip_blank_lines = True, engine = 'python', skiprows = 18000) 
            #Give the FF columns the same headers as other files
            df.columns = ['Date and Time', 'Pressure (mmHg)', 'Temperature (C)','Depth to Water (mm)'] 
            #remove na values
            idx = df.loc[pd.isna(df["Pressure (mmHg)"]), :].index
            df = df.drop(idx)
        else:
            df = pd.read_csv('CSV//'+f, skip_blank_lines = True, engine = 'python')
        # Convert datetime to datetime index to speed things up and enable better string parsing
        # Round to nearest hour to match barometer data
        analysisDict[f]['Datetime'] = pd.DatetimeIndex(df['Date and Time'],dayfirst=True).round("H")
        #Deal with different sites and merge barometer data
        analysisDict[f].loc[:,'ID'] = uqid
        analysisDict[f].loc[:,'Site'] = site
        analysisDict[f].loc[:,'Region'] = region
        if site == 'MH':
            analysisDict[f].loc[:,'B'] = B[site][region] 
            analysisDict[f].loc[:,'C'] = C[site]
            analysisDict[f] = analysisDict[f].merge(MH_Baro, left_on='Datetime', right_on='Datetime', how = 'left')     
        if site == 'Mig':
            analysisDict[f].loc[:,'B'] = B[site]
            analysisDict[f].loc[:,'C'] = C[site][region+pos]
            analysisDict[f] = analysisDict[f].merge(Mig_Baro, left_on='Datetime', right_on='Datetime', how = 'left')      
        if site == 'FF':
            analysisDict[f].loc[:,'B'] = B[site][pos]
            analysisDict[f].loc[:,'C'] = C[site][pos]
            analysisDict[f] = analysisDict[f].merge(FF_Baro, left_on='Datetime', right_on='Datetime', how = 'left')     
        # Process the datetime into different time measures
        analysisDict[f]['Date'] = analysisDict[f]['Datetime'].dt.date #extract date
        analysisDict[f]['Hour'] = analysisDict[f]['Datetime'].dt.hour #hour
        analysisDict[f]['Day'] = analysisDict[f]['Datetime'].dt.dayofyear #day of year
        analysisDict[f]['Week'] = analysisDict[f]['Datetime'].dt.isocalendar().week.astype(int) #week
        analysisDict[f]['Month'] = analysisDict[f]['Datetime'].dt.month# month
        analysisDict[f]['Year'] = analysisDict[f]['Datetime'].dt.year #and yearly values
        analysisDict[f]['Quarter'] = analysisDict[f]['Datetime'].dt.quarter #quarterly value
        # Calculate WTD 
        analysisDict[f].loc[:,'Temperature(Logger)'] = df.loc[:,'Temperature (C)'].astype(float)
        analysisDict[f].loc[:,'mmHg(Logger)'] = df.loc[:,'Pressure (mmHg)'].astype(float)
        analysisDict[f].loc[:,'mmHg(Logger)'] = df.loc[:,'Pressure (mmHg)'].astype(float)
        analysisDict[f].loc[:,'mmH2O'] = (analysisDict[f].loc[:,'mmHg(Logger)'] -  analysisDict[f].loc[:,'mmHg(Baro)']) * HG_TO_H2O
        analysisDict[f].loc[:,'A'] = A
        analysisDict[f].loc[:,'E'] = analysisDict[f].loc[:,'C'] - analysisDict[f].loc[:,'B'] + analysisDict[f].loc[:,'A']
        analysisDict[f].loc[:,'WTD(mm)'] = analysisDict[f].loc[:,'E'] -  analysisDict[f].loc[:,'mmH2O']
        #reorder the columns
        analysisDict[f] = analysisDict[f][columnOrder]
        #write to csv files
        analysisDict[f].to_csv(f'EXPORT//p_{f}', index = False)

Merge all individual datasets into one 

In [7]:
mergeDf = pd.DataFrame(columns=columnOrder)
#Merge the barometer data into a single datafram
for df in analysisDict.values():
    mergeDf = pd.merge(mergeDf,df, how = 'outer')
mergeDf.to_csv('EXPORT//p_mergedData.csv', index = False)

In [8]:
mergeDf

,ID,Site,Region,Datetime,Date,Hour,Day,Week,Month,Quarter,...,Temperature(Logger),Temperature(Baro),mmHg(Logger),mmHg(Baro),mmH2O,A,B,C,E,WTD(mm)
0,FFDAMA,FF,DAM,2021-06-29 09:00:00,2021-06-29,9,180,26,6,2,...,8.384857,11.83365,796.8083,748.9008,651.307253,20,63,980,937,285.692747
1,FFDAMA,FF,DAM,2021-06-29 10:00:00,2021-06-29,10,180,26,6,2,...,8.384857,13.38490,797.2664,749.0449,655.576115,20,63,980,937,281.423885
2,FFDAMA,FF,DAM,2021-06-29 11:00:00,2021-06-29,11,180,26,6,2,...,8.384857,14.25948,797.3047,749.2086,653.871289,20,63,980,937,283.128711
3,FFDAMA,FF,DAM,2021-06-29 12:00:00,2021-06-29,12,180,26,6,2,...,8.384857,15.07967,797.3429,749.3637,652.282022,20,63,980,937,284.717978
4,FFDAMA,FF,DAM,2021-06-29 13:00:00,2021-06-29,13,180,26,6,2,...,8.384857,16.50833,797.1520,749.5903,646.606068,20,63,980,937,290.393932
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166868,MigRES2,Mig,RES,2023-02-23 08:00:00,2023-02-23,8,54,8,2,1,...,6.210000,0.78400,796.8200,725.4470,970.323072,20,50,1010,980,9.676928
166869,MigRES2,Mig,RES,2023-02-23 09:00:00,2023-02-23,9,54,8,2,1,...,6.049000,1.82200,796.8360,726.1530,960.942453,20,50,1010,980,19.057547
166870,MigRES2,Mig,RES,2023-02-23 10:00:00,2023-02-23,10,54,8,2,1,...,6.049000,4.36600,796.7580,727.0030,948.326200,20,50,1010,980,31.673800
166871,MigRES2,Mig,RES,2023-02-23 11:00:00,2023-02-23,11,54,8,2,1,...,NaN,4.41900,NaN,726.8410,NaN,20,50,1010,980,NaN
